### Phishing URL Website detection using Machine Learning

##### Problem Statement
The problem is to detect whether a URL is phishing or legitimate using Machine Learning. The phishing URLs looks like legitimate but redirects to a malicious website. So, our goal is to build a detection system that detects any type of URL using Machine Learning with low latency.

URLs are taken from different data sources and combined together. By carefully observing those URLs, the common patterns I found are
- Some of the Phishing URLs have HTTP protocol. This may be due to the ease of setup and less secure. But most of the Phishing Websites have HTTPS protocol overcoming the security.
- Rarely FTP protocol is used to download malicious content by clicking on the URL.
- Many of the Phishing URLs use IP address as the domain name. This sometimes make the users harder to identify the site as fradulent.
- Phishing URLs length is very larger. Many characters are present in Path, Domain and Query.
- Phishing URLs embed hexadecimal characters to escape detection by security.

In [2]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv('data/raw/raw_data.csv')

data.head()

,url,label
0,https://adoaeco.cn/Loggin,phishing
1,https://gageparkhighschool.com/QTeuUe,phishing
2,https://wwnox.miraltek.cfd/qzxn3,phishing
3,https://halfetitur.com/?token=r2I0IU0FEHfPf5Dn,phishing
4,https://yqcjl.miraltek.cfd/plis0,phishing


In [4]:
print(f'The dataset consists of {data.shape[0]} rows and {data.shape[1]} columns')

The dataset consists of 253098 rows and 2 columns


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253098 entries, 0 to 253097
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   url     253098 non-null  object
 1   label   253098 non-null  object
dtypes: object(2)
memory usage: 3.9+ MB


In [6]:
data.describe()

,url,label
count,253098,253098
unique,253098,2
top,https://adoaeco.cn/Loggin,legitimate
freq,1,129420


**Cleaning the URLs**

In [7]:
import re

def clean_url(url):

    # Remove zero-width characters
    zero_width = r'[\u200B\u200C\u200D\u2060\uFEFF]'
    url = re.sub(zero_width, '', url)

    # Remove all ASCII control characters (0–31 and 127)
    url = re.sub(r'[\x00-\x1F\x7F]', '', url)

    # Remove non-printable Unicode control symbols
    unicode_controls = r'[\u202A-\u202E\u2066-\u2069]'
    url = re.sub(unicode_controls, '', url)

    # Remove soft hyphens & BOM issues
    url = url.replace('\u00AD', '').replace('\ufeff', '')

    # Normalize accidental triple slashes
    url = re.sub(r':\/\/\/+', '://', url)

    # Fix missing colon after https (caused by invisible chars earlier)
    url = re.sub(r'^(https?)(\/\/)', r'\1://', url)

    # Remove spaces around colon in scheme (malformed URLs)
    url = re.sub(r'^(https?)\s*:\s*\/\/', r'\1://', url)

    # Remove trailing slashes
    url = url.rstrip('/')

    return url

data['url'] = data['url'].apply(clean_url)

**Checking null & duplicate values**

In [8]:
data.isnull().sum()

url      0
label    0
dtype: int64

In [9]:
data.duplicated().sum()

np.int64(0)

**Validating URLs**

In [10]:
url_pattern = r"^(([^:/?#]+):)?(//([^/?#]*))?([^?#]*)(\?([^#]*))?(#(.*))?"

print(sum(data['url'].str.match(url_pattern)))

253098


In [11]:
data.to_csv('data/transformed/cleaned_raw_data.csv',index=False)

On carefully observing the data, I found that dataset contains, many shorten URLs. Some shorten URLs may work and some may not. The working shortened URLs are expanded by using a cache of shortening sub-domains and async functions. 

In [12]:
# dataset consists of unshorten URLs
df = pd.read_csv('data/raw/expanded_urls.csv')
df.head()

,url,label,expanded_url
0,https://adoaeco.cn/Loggin,phishing,https://adoaeco.cn/Loggin
1,https://gageparkhighschool.com/QTeuUe,phishing,https://gageparkhighschool.com/QTeuUe
2,https://wwnox.miraltek.cfd/qzxn3,phishing,https://wwnox.miraltek.cfd/qzxn3
3,https://halfetitur.com/?token=r2I0IU0FEHfPf5Dn,phishing,https://halfetitur.com/?token=r2I0IU0FEHfPf5Dn
4,https://yqcjl.miraltek.cfd/plis0,phishing,https://yqcjl.miraltek.cfd/plis0


In [13]:
sum(df['url'] != df['expanded_url'])

128

As many of the shortened URLs are not working properly, only some shortened URLs are expanded.

In [14]:
def replace_shortened_url(url,expanded_url):
    if url == expanded_url:
        return url
    return expanded_url

data['url'] = df.apply(lambda row: replace_shortened_url(row['url'],row['expanded_url']),axis=1)

In [15]:
data.head()

,url,label
0,https://adoaeco.cn/Loggin,phishing
1,https://gageparkhighschool.com/QTeuUe,phishing
2,https://wwnox.miraltek.cfd/qzxn3,phishing
3,https://halfetitur.com/?token=r2I0IU0FEHfPf5Dn,phishing
4,https://yqcjl.miraltek.cfd/plis0,phishing


In [16]:
data.isnull().sum()

url      0
label    0
dtype: int64

In [17]:
data.duplicated(subset=['url']).sum()

np.int64(47)

In [18]:
data[data.duplicated(subset=['url'])]

,url,label
4138,https://scanned.page/p/3jY5UN?id=mt8YdgnKiZ,phishing
4139,https://scanned.page/p/3OhzCy?id=nkuyv79qJ7,phishing
4216,https://scanned.page/p/3jY5UN?id=O5rlEUgEjg,phishing
4244,https://scanned.page/p/3OhzCy?id=vNPx5qXIDe,phishing
4889,https://parkingtreeco.com/wp-content/autopas/1...,phishing
5129,https://did.li/secure/,phishing
5325,https://did.li/secure/,phishing
7040,https://did.li/secure/,phishing
7813,https://did.li/secure/,phishing
8131,https://did.li/secure/,phishing


In [19]:
data.drop_duplicates(subset=['url'],inplace=True,ignore_index=True)

In [20]:
data.duplicated().sum()

np.int64(0)

In [21]:
data.shape

(253051, 2)

**Extracting URL Components** : These elements represent the structural building blocks of a URL and are useful for detailed analysis & feature engineering.
- Protocol/Scheme of the URL
- Domain of the URL
- Path of the URL
- Query of the URL
- TLD of the URL
- SLD of the URL
- Path of the URL
- Query of the URL

Consider the below URL as an example:
https://www.google.com/search/name=phishing_url

- Scheme/Protocol: `https`
- Sub-domain: `www`
- Domain name: `google.com`
- Second-Level Domain: `google`
- Top-Level Domain: `com`
- Path: `search`
- Query: `name=phishing_url`

In [22]:
url_components_df = data[['url','label']]

In [23]:
# Extracting protocol, domain, path, query, sld, tld & subdomain
from urllib.parse import urlsplit
import tldextract

def parse_url(url):
    try:
        result = urlsplit(url)
        protocol = result.scheme
        domain   = result.netloc
        path     = result.path
        query    = result.query
        ext = tldextract.extract(domain)
        subdomain = ext.subdomain
        sld = ext.domain
        tld = ext.suffix
    except Exception:   # Handles IPv6 Address
        protocol = ""
        domain = ""
        path = ""
        query = ""
        sld = ""
        tld = ""
        subdomain = ""

    return protocol, domain, subdomain, tld, sld, path, query

url_components_df[['protocol','domain','subdomain','tld','sld','path','query']] = data['url'].apply(parse_url).apply(pd.Series)

In [24]:
# Handling IP-based URLs
url_components_df[url_components_df['domain'].str.fullmatch(r'\d{1,3}(\.\d{1,3}){3}')]

,url,label,protocol,domain,subdomain,tld,sld,path,query
2994,https://91.92.241.186,phishing,https,91.92.241.186,,,91.92.241.186,,
3667,https://140.99.164.68/x0,phishing,https,140.99.164.68,,,140.99.164.68,/x0,
6300,https://31.172.87.101/x0,phishing,https,31.172.87.101,,,31.172.87.101,/x0,
14536,https://43.153.99.18,phishing,https,43.153.99.18,,,43.153.99.18,,
16733,https://185.187.56.126,phishing,https,185.187.56.126,,,185.187.56.126,,
...,...,...,...,...,...,...,...,...,...
252640,http://67.23.255.34/~comentar/Face/Facebook_Co...,phishing,http,67.23.255.34,,,67.23.255.34,/~comentar/Face/Facebook_Comentario.exe,
252716,http://191.101.7.221/fire/aasdqwe,phishing,http,191.101.7.221,,,191.101.7.221,/fire/aasdqwe,
252961,http://178.217.186.224/panel/etc/info/toke/cp....,phishing,http,178.217.186.224,,,178.217.186.224,/panel/etc/info/toke/cp.php,m=login
252990,http://185.75.46.73/information.cgi,phishing,http,185.75.46.73,,,185.75.46.73,/information.cgi,


In [25]:
ip_idx = url_components_df[url_components_df['sld'].str.fullmatch(r'\d{1,3}(\.\d{1,3}){3}')].index

url_components_df['is_ip'] = False
url_components_df.iloc[ip_idx,-1] = True
url_components_df.iloc[ip_idx,[3,6]] = ''

url_components_df.iloc[ip_idx]

,url,label,protocol,domain,subdomain,tld,sld,path,query,is_ip
2994,https://91.92.241.186,phishing,https,,,,,,,True
3667,https://140.99.164.68/x0,phishing,https,,,,,/x0,,True
6300,https://31.172.87.101/x0,phishing,https,,,,,/x0,,True
14536,https://43.153.99.18,phishing,https,,,,,,,True
16733,https://185.187.56.126,phishing,https,,,,,,,True
...,...,...,...,...,...,...,...,...,...,...
252716,http://191.101.7.221/fire/aasdqwe,phishing,http,,,,,/fire/aasdqwe,,True
252811,http://91.239.25.38:6892,phishing,http,,,,,,,True
252961,http://178.217.186.224/panel/etc/info/toke/cp....,phishing,http,,,,,/panel/etc/info/toke/cp.php,m=login,True
252990,http://185.75.46.73/information.cgi,phishing,http,,,,,/information.cgi,,True


In [26]:
url_components_df.head()

,url,label,protocol,domain,subdomain,tld,sld,path,query,is_ip
0,https://adoaeco.cn/Loggin,phishing,https,adoaeco.cn,,cn,adoaeco,/Loggin,,False
1,https://gageparkhighschool.com/QTeuUe,phishing,https,gageparkhighschool.com,,com,gageparkhighschool,/QTeuUe,,False
2,https://wwnox.miraltek.cfd/qzxn3,phishing,https,wwnox.miraltek.cfd,wwnox,cfd,miraltek,/qzxn3,,False
3,https://halfetitur.com/?token=r2I0IU0FEHfPf5Dn,phishing,https,halfetitur.com,,com,halfetitur,/,token=r2I0IU0FEHfPf5Dn,False
4,https://yqcjl.miraltek.cfd/plis0,phishing,https,yqcjl.miraltek.cfd,yqcjl,cfd,miraltek,/plis0,,False


**URL Length Features** : These metrics help capture patterns that may be indicative of URL behaviour or intent.
- Length of the URL
- Length of the Domain
- Length of the Path
- Length of the Query
- URL Depth (no. of '/' segments in path)
- No. of Subdomains

In [27]:
url_len_features_df = data[['url','label']]

In [28]:
# Length of the URL
url_len_features_df['url_len'] = data['url'].str.len()

In [29]:
# Length of the Domain
url_len_features_df['domain_len'] = url_components_df['domain'].str.len()

In [30]:
# Length of the Path
def path_length(path):
    if not isinstance(path,str) or not path.strip():
        return 0
    
    if path.startswith('/'):
        path = path[1:]
    
    segments = [seg for seg in path.split('/') if seg]

    return sum(len(seg) for seg in segments)

url_len_features_df['path_len'] = url_components_df['path'].apply(path_length)

In [31]:
# Length of the Query
url_len_features_df['query_len'] = url_components_df['query'].str.len()

In [32]:
# Depth of the URL
protocol_re = re.compile(r'^[a-zA-Z][a-zA-Z0-9+.-]*:(//)?')

def calculate_url_depth(url):
    url = protocol_re.sub('',url)
    parts = url.split('/',1)

    if len(parts) == 1:
        return 0
    path = parts[1]

    depth = sum(1 for seg in path.split('/') if seg)

    return depth

url_len_features_df['url_depth'] = data['url'].apply(calculate_url_depth)

In [33]:
# Subdomain count
url_len_features_df['subdomain_count'] = url_components_df['subdomain'].str.split('.').apply(lambda x: len(x))
url_len_features_df.loc[ip_idx,'subdomain_count'] = 0   # IP-based URLs

In [34]:
url_len_features_df.head()

,url,label,url_len,domain_len,path_len,query_len,url_depth,subdomain_count
0,https://adoaeco.cn/Loggin,phishing,25,10,6,0,1,1
1,https://gageparkhighschool.com/QTeuUe,phishing,37,22,6,0,1,1
2,https://wwnox.miraltek.cfd/qzxn3,phishing,32,18,5,0,1,1
3,https://halfetitur.com/?token=r2I0IU0FEHfPf5Dn,phishing,46,14,0,22,1,1
4,https://yqcjl.miraltek.cfd/plis0,phishing,32,18,5,0,1,1


**Domain Structure Features** : These features may provide insights into domain structure and potential indicators of unusual or suspicious patterns.
- TLD Length
- Is the domain a IP address?
- Does the URL contains Port number?

In [35]:
domain_features_df = data[['url','label']]
domain_features_df['tld'] = url_components_df[['tld']]

In [36]:
# TLD Length
domain_features_df['tld_len'] = domain_features_df['tld'].str.len()

In [37]:
# Is the domain a IPv4 address?
domain_features_df['url_has_ipv4'] = url_components_df['is_ip']

In [38]:
# Does the URL contains Port number?
from urllib.parse import urlparse

def url_has_port(url):
    try:
        return urlparse(url).port is not None
    except Exception:
        return False

domain_features_df['url_has_port'] = url_components_df['url'].apply(url_has_port)

In [39]:
domain_features_df.head()

,url,label,tld,tld_len,url_has_ipv4,url_has_port
0,https://adoaeco.cn/Loggin,phishing,cn,2,False,False
1,https://gageparkhighschool.com/QTeuUe,phishing,com,3,False,False
2,https://wwnox.miraltek.cfd/qzxn3,phishing,cfd,3,False,False
3,https://halfetitur.com/?token=r2I0IU0FEHfPf5Dn,phishing,com,3,False,False
4,https://yqcjl.miraltek.cfd/plis0,phishing,cfd,3,False,False


**SLD-based Features** : These features carries meaningful information about the entity represented by the URL.
- Length of SLD
- SLD has digit?
- SLD has hyphen?
- SLD Token count

In [40]:
sld_features_df = data[['url','label']]
sld_features_df['sld'] = url_components_df[['sld']]

In [41]:
# Length of SLD
sld_features_df['sld_len'] = sld_features_df['sld'].str.len()

In [42]:
# SLD has digit?
sld_features_df['sld_has_digit'] = sld_features_df['sld'].str.contains(r'\d')

In [43]:
# SLD has hyphen?
sld_features_df['sld_has_hyphen'] = sld_features_df['sld'].str.contains('-')

In [44]:
# No of tokens in SLD
sld_features_df['sld_token_count'] = (sld_features_df['sld'].str.split('-')).str.len()

In [45]:
sld_features_df.head()

,url,label,sld,sld_len,sld_has_digit,sld_has_hyphen,sld_token_count
0,https://adoaeco.cn/Loggin,phishing,adoaeco,7,False,False,1
1,https://gageparkhighschool.com/QTeuUe,phishing,gageparkhighschool,18,False,False,1
2,https://wwnox.miraltek.cfd/qzxn3,phishing,miraltek,8,False,False,1
3,https://halfetitur.com/?token=r2I0IU0FEHfPf5Dn,phishing,halfetitur,10,False,False,1
4,https://yqcjl.miraltek.cfd/plis0,phishing,miraltek,8,False,False,1


**Character Features** : These features may help to reveal obfuscation techniques in phishing URLs.
- Dot count in Domain
- Hyphen count in domain + path
- Underscore count in path + query
- Slash count in URL
- Digit count in URL
- Alphabet count in URL

In [46]:
char_features_df = data[['url','label']]

In [47]:
# Dot count in Domain
char_features_df['dot_count_domain'] = url_components_df['domain'].str.count(r'\.')

In [48]:
# Hyphen count in domain + path
char_features_df['hyphen_count_domain_path'] = (url_components_df['domain'] + url_components_df['path']).str.count('-')

In [49]:
# Underscore count in path + query
char_features_df['underscore_count_path_query'] = (url_components_df['path'] + url_components_df['query']).str.count('\_')

In [50]:
# Slash count in URL
char_features_df['slash_count'] = data['url'].str.count('/')

In [51]:
# Digit count in URL
char_features_df['digit_count'] = data['url'].str.count(r'\d')

In [52]:
# Alphabet count in URL
char_features_df['alphabet_count'] = data['url'].str.count(r'[a-zA-Z]')

In [53]:
# Special characters count in URL
char_features_df['spl_char_count'] = data['url'].str.count(r'[^a-zA-Z0-9]')

In [54]:
char_features_df.head()

,url,label,dot_count_domain,hyphen_count_domain_path,underscore_count_path_query,slash_count,digit_count,alphabet_count,spl_char_count
0,https://adoaeco.cn/Loggin,phishing,1,0,0,3,0,20,5
1,https://gageparkhighschool.com/QTeuUe,phishing,1,0,0,3,0,32,5
2,https://wwnox.miraltek.cfd/qzxn3,phishing,2,0,0,3,1,25,6
3,https://halfetitur.com/?token=r2I0IU0FEHfPf5Dn,phishing,1,0,0,3,4,35,7
4,https://yqcjl.miraltek.cfd/plis0,phishing,2,0,0,3,1,25,6


**Entropy features** : Higher entropy often indicates obfuscation or automatically generated strings, which may be strong signals in detecting phishing URLs.
- Entropy of URL
- Entropy of Domain
- Entropy of SLD
- Entropy of Path

In [55]:
# Entropy calculation
import math
from collections import Counter

def shannon_entropy(str_):
    counts = Counter(str_)
    length = len(str_)

    return -sum((count / length) * math.log2(count / length) for count in counts.values())

In [56]:
entropy_feature_df = data[['url','label']]

In [57]:
# Entropy of URL
entropy_feature_df['url_entropy'] = data['url'].apply(shannon_entropy)

In [58]:
# Entropy of Domain
entropy_feature_df['domain_entropy'] = url_components_df['domain'].apply(shannon_entropy)

In [59]:
# Entropy of SLD
entropy_feature_df['sld_entropy'] = url_components_df['sld'].apply(shannon_entropy)

In [60]:
# Entropy of Path
entropy_feature_df['path_entropy'] = url_components_df['path'].apply(shannon_entropy)

In [61]:
entropy_feature_df.head()

,url,label,url_entropy,domain_entropy,sld_entropy,path_entropy
0,https://adoaeco.cn/Loggin,phishing,3.863465,2.721928,2.235926,2.521641
1,https://gageparkhighschool.com/QTeuUe,phishing,4.208925,3.629220,3.419382,2.521641
2,https://wwnox.miraltek.cfd/qzxn3,phishing,4.452820,3.947703,3.000000,2.584963
3,https://halfetitur.com/?token=r2I0IU0FEHfPf5Dn,phishing,4.760096,3.664498,3.121928,-0.000000
4,https://yqcjl.miraltek.cfd/plis0,phishing,4.241729,3.836592,3.000000,2.584963


**Token based Features** : Tokenization helps revealing structural patterns and semantic clues within the URL.
- Token count in domain
- Token count in path
- Token Count in domain + path
- Avg Token Length

In [62]:
token_features_df = data[['url','label']]

In [63]:
# Calculating Token count in path + domain
def count_tokens(domain,path):
    try:    
        domain_tokens = domain.replace('.',' ').replace('-',' ').replace('_',' ').split()
        path_tokens = path.replace('.',' ').replace('-',' ').replace('_',' ').split()

        total_tokens = len(domain_tokens) + len(path_tokens)

        return len(domain_tokens), len(path_tokens), total_tokens
    
    except Exception:
        return 0,0,0
    
token_features_df[['domain_token_count','path_token_count','total_tokens']] = url_components_df.apply(lambda row: count_tokens(row['domain'],row['path']),axis=1).apply(pd.Series)

In [64]:
# Average token length of the URL
protocol_re = re.compile(r'^[a-zA-Z][a-zA-Z0-9+.-]*://',re.IGNORECASE)
split_re = re.compile(r'[./?=\-_&:%]+')

def avg_token_length(url):
    url = protocol_re.sub('',url)
    tokens = split_re.split(url)
    tokens = [t for t in tokens if t]

    if not tokens:
        return 0
    
    return sum(len(t) for t in tokens) / len(tokens)

token_features_df['avg_token_length'] = data['url'].apply(avg_token_length)

In [65]:
token_features_df.head()

,url,label,domain_token_count,path_token_count,total_tokens,avg_token_length
0,https://adoaeco.cn/Loggin,phishing,2,1,3,5.00
1,https://gageparkhighschool.com/QTeuUe,phishing,2,1,3,9.00
2,https://wwnox.miraltek.cfd/qzxn3,phishing,3,1,4,5.25
3,https://halfetitur.com/?token=r2I0IU0FEHfPf5Dn,phishing,2,1,3,8.50
4,https://yqcjl.miraltek.cfd/plis0,phishing,3,1,4,5.25


**Hexadecimal based Features**
- URL has Hexadecimal Characters
- No. of Hexadecimal Characters
- Hexadecimal Ratio

In [66]:
hex_feature_df = data[['url','label']]

In [67]:
# URL has Hexadecimal Characters?
hex_feature_df['has_hex'] = data['url'].str.contains(r'%[0-9A-Fa-f]{2}')

In [68]:
# No. of hexadecimal characters

def count_hex_chars(url):
    count = 0
    i = 0
    while i < len(url):
        if url[i] == '%' and i+2 < len(url):
            c1, c2 = url[i+1], url[i+2]

            if (c1.isdigit() or c1.lower() in 'abcdef') and (c2.isdigit() or c2.lower() in 'abcdef'):
                count += 2
                i += 3
                continue
        i += 1

    return count

hex_feature_df['hex_char_count'] = data['url'].apply(count_hex_chars)

In [69]:
# Hexadecimal Ratio
hex_feature_df['hex_ratio'] = hex_feature_df['hex_char_count'] / url_len_features_df['url_len']

In [70]:
hex_feature_df.head()

,url,label,has_hex,hex_char_count,hex_ratio
0,https://adoaeco.cn/Loggin,phishing,False,0,0.0
1,https://gageparkhighschool.com/QTeuUe,phishing,False,0,0.0
2,https://wwnox.miraltek.cfd/qzxn3,phishing,False,0,0.0
3,https://halfetitur.com/?token=r2I0IU0FEHfPf5Dn,phishing,False,0,0.0
4,https://yqcjl.miraltek.cfd/plis0,phishing,False,0,0.0


In [71]:
url_components_df.drop(columns=['is_ip'],inplace=True)

In [72]:
# Saving all the extracted features in the corresponding CSV files.

data.to_csv('data/transformed/final_raw_data.csv',index=False)
url_components_df.to_csv('data/transformed/1.url_components_data.csv',index=False)
url_len_features_df.to_csv('data/transformed/2.component_len_features_data.csv',index=False)
domain_features_df.to_csv('data/transformed/3.domain_features_data.csv',index=False)
sld_features_df.to_csv('data/transformed/4.sld_features_data.csv',index=False)
char_features_df.to_csv('data/transformed/5.char_features_data.csv',index=False)
entropy_feature_df.to_csv('data/transformed/6.entropy_feature_data.csv',index=False)
token_features_df.to_csv('data/transformed/7.token_features_data.csv',index=False)
hex_feature_df.to_csv('data/transformed/8.hex_features_data.csv',index=False)